In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx

import requests
import json
from six.moves.urllib.request import urlopen
from overpass import API

In [15]:
graph = ox.graph_from_place("Ile de France", custom_filter = '[highway][highway != "service"][highway != "footway"][highway != "motorway"][highway != "motorway_link"][bicycle != "no"][highway != "steps"][highway != "busway"][highway != "pedestrian"]')

In [ ]:
import pickle

# save graph object t  o file
pickle.dump(graph, open('paris_graph.pickle', 'wb'))

In [78]:
def get_pos_from_street(name: str, city:str):
    strForQuery = name.replace(' ', '+')
    url = 'https://nominatim.openstreetmap.org/search?q=' + strForQuery + f'%2C+{city}&format=geojson'
    query = json.loads(urlopen(url).read())

    return query['features'][0]['geometry']['coordinates']



print(f"Pos = {get_pos_from_street('93 Rue de la Chapelle', 'Paris')}")

Pos = [2.3586706, 48.8979868]


In [92]:
# Adresse de départ et adresse d'arrivée
start_address = "93 Rue de la Chapelle, Paris"

# Récupération des coordonnées de départ et d'arrivée
start_point = ox.geocode(start_address)

graph = ox.graph_from_point(start_point, dist=50, simplify=False)
node = ox.nearest_nodes(graph, start_point[1], start_point[0])

voisins = graph[node]

for voisin in voisins:
    print(f'Node id = {voisin}')
    print(f'Edges = {voisins[voisin][0]["osmid"]} Length = ', voisins[voisin][0]['length'])

Node id = 6659097297
Edges = 1135730432 Length =  24.511
Node id = 10586970470
Edges = 1135730433 Length =  7.961
{6659097297: {0: {'osmid': 1135730432, 'highway': 'service', 'maxspeed': '20', 'service': 'driveway', 'access': 'private', 'oneway': False, 'reversed': True, 'length': 24.511}}, 10586970470: {0: {'osmid': 1135730433, 'tunnel': 'building_passage', 'highway': 'service', 'maxspeed': '20', 'service': 'driveway', 'access': 'private', 'oneway': False, 'reversed': False, 'length': 7.961}}}


In [123]:
def get_node_pos_from_id(nodeId: int):
    api = API(endpoint='http://localhost/api/interpreter')
    pos = api.get(f'node(id:{nodeId});', responseformat="csv(::lon,::lat)")
    print(pos)

    return float(pos[1][1]), float(pos[1][0])

def get_neighborhood_at(nodeId: int):

    lon, lat = get_node_pos_from_id(nodeId)

    graph = ox.graph_from_point((lon, lat), dist=125, simplify=False, custom_filter = '[highway][highway != "service"][highway != "footway"][highway != "motorway"][highway != "motorway_link"][bicycle != "no"][highway != "steps"][highway != "busway"][highway != "pedestrian"]')

    neighbors = graph[nodeId]
    nodes = []
    edges = []

    for voisin in neighbors:
        if voisin != 0:
            nodes.append(voisin)
            edges.append(neighbors[voisin][0]["osmid"])

    return nodes, edges

nodes, edges = get_neighborhood_at(1762073410)

print('Nodes : ', nodes)
print('Edges : ', edges)

[['@lon', '@lat'], ['2.3591416', '48.8971056']]
Nodes :  [8165345385, 5240597749, 5240597743]
Edges :  [53351901, 542019795, 544727632]


In [126]:
current_node = 5240597749
unwanted_num = {-1}

for i in range(50):
    nodes, edges = get_neighborhood_at(current_node)

    print('Nodes : ', nodes)
    print('Edges : ', edges)

    current_node = nodes[-1]
    get_edge_from_id(edges[-1])

[['@lon', '@lat'], ['2.3592927', '48.8971100']]
Nodes :  [5240597745, 5915269269, 1762073410]
Edges :  [164588919, 245453106, 542019795]
{"features": [{"geometry": {"coordinates": [], "type": "LineString"}, "id": 542019795, "properties": {"highway": "residential", "maxspeed": "30", "name": "Rue du Pré"}, "type": "Feature"}], "type": "FeatureCollection"}
C'est une mauvaise ou bonne route !
[['@lon', '@lat'], ['2.3591416', '48.8971056']]
Nodes :  [8165345385, 5240597749, 5240597743]
Edges :  [53351901, 542019795, 544727632]
{"features": [{"geometry": {"coordinates": [], "type": "LineString"}, "id": 544727632, "properties": {"highway": "primary", "lanes": "3", "lit": "yes", "maxspeed": "30", "name": "Rue de la Chapelle", "oneway": "yes", "surface": "asphalt"}, "type": "Feature"}], "type": "FeatureCollection"}
C'est une mauvaise ou bonne route !
[['@lon', '@lat'], ['2.3591473', '48.8970251']]
Nodes :  [5240597744]
Edges :  [544727632]
{"features": [{"geometry": {"coordinates": [], "type": 

In [21]:
def get_graph_at(nodeId: int, surface: int):
    lon, lat = get_node_pos_from_id(nodeId)

    graph = ox.graph_from_point((lon, lat), dist=surface, simplify=True, custom_filter = '[highway][highway != "service"][highway != "footway"][highway != "motorway"][highway != "motorway_link"][bicycle != "no"][highway != "steps"][highway != "busway"][highway != "pedestrian"]')

    return graph

In [2]:
def is_edge_cycleway(edgeInfo):
    props = edgeInfo[0]['properties']
    highway = props['highway']

    if 'bicycle' in props:
        if props['bicycle'] == 'designated' or props['bicycle']  == 'yes':
            return True

    if highway == 'cycleway':
        return True

    return False

def is_edge_shared_cycleway(edgeInfo):
    props = edgeInfo[0]['properties']

    if 'cycleway:right' in props:
        return True

    return False

def is_segregated(edgeInfo):
    props = edgeInfo[0]['properties']

    if 'segregated' in props:
        if props['segregated'] == 'yes':
            return True

    return False

def get_edge_speed(edgeInfo):
    props = edgeInfo[0]['properties']

    if 'maxspeed' in props:
        return props['maxspeed']
    
    return None

def get_edge_from_id(edgeId: int, api):
    edgeInfo = api.get(f'way(id:{edgeId});')
    
    isCycleway = is_edge_cycleway(edgeInfo)
    maxSpeed = get_edge_speed(edgeInfo)

    props = edgeInfo[0]['properties']
    highway = props['highway']

    if highway == 'residential':
        return [1, 0, 0, 0]
    
    if isCycleway:
        if is_segregated(edgeInfo):
            return [1, 0, 0, 0]
        else:
            return [1, 1, 0, 0]
    elif maxSpeed != None and int(maxSpeed) <= 50:
        if is_edge_shared_cycleway(edgeInfo):
            return [1, 1, 0, 0]
        else:
            return [1, 1, 1, 0]
    else:
        return [1, 1, 1, 1]

# Convertir le graphe osmnx/pickle avec les bons poids.

In [3]:
import pickle
import networkx as nx
from overpass import API

In [6]:
graph = ox.graph_from_place("Paris", custom_filter = '[highway][highway != "service"][highway != "footway"][highway != "motorway"][highway != "motorway_link"][bicycle != "no"][highway != "steps"][highway != "busway"][highway != "pedestrian"]')

In [13]:
import pickle

# save graph object t  o file
graph = pickle.load(open('paris_graph.pickle', 'rb'))

print(graph)

MultiDiGraph with 13587 nodes and 29094 edges


In [7]:
def get_pos_at(osm_id, api):
    pos = api.get(f'node(id:{osm_id});', responseformat="csv(::lon,::lat)")

    return pos[1]

In [8]:
import numpy as np

In [9]:
G = nx.Graph()
already_exists = []

api = API(endpoint='http://localhost/api/interpreter')

i = 0

for edge in graph.edges:
    try:
        i += 1
        if i % 100 == 0:
            print(f'Arête n°{i} en cours de génération...')
        n1 = edge[0]
        n2 = edge[1]
        edgeId = graph[n1][n2][0]['osmid']
        #Ajout des noeuds.
        pos = get_pos_at(n1, api)
        G.add_node(n1, lon=pos[1], lat=pos[0])
        pos = get_pos_at(n2, api)
        G.add_node(n2, lon=pos[1], lat=pos[0])

        #Ajout de l'arête en question dans le graphe.
        if isinstance(edgeId, list):
            weights = np.array(get_edge_from_id(edgeId[0], api))
            G.add_edge(n1, n2, weight=weights * graph[n1][n2][0]['length'])
        else:
            weights = np.array(get_edge_from_id(edgeId, api))
            G.add_edge(n1, n2, weight=weights * graph[n1][n2][0]['length'])
    except:
        print('Erreur avec : ', graph[n1][n2][0])

Arête n°100 en cours de génération...
Arête n°200 en cours de génération...
Erreur avec :  {'osmid': 1156065410, 'oneway': True, 'name': "Place d'Italie", 'highway': 'cycleway', 'width': '1', 'reversed': False, 'length': 12.097}
Arête n°300 en cours de génération...
Arête n°400 en cours de génération...
Arête n°500 en cours de génération...
Erreur avec :  {'osmid': [813587017, 642898830], 'lanes': '8', 'name': 'Avenue des Champs-Élysées', 'highway': 'primary', 'maxspeed': '50', 'oneway': False, 'reversed': False, 'length': 98.19900000000001, 'geometry': <shapely.geometry.linestring.LineString object at 0x00000229A7D6E3D0>}
Arête n°600 en cours de génération...
Arête n°700 en cours de génération...
Arête n°800 en cours de génération...
Arête n°900 en cours de génération...
Arête n°1000 en cours de génération...
Erreur avec :  {'osmid': 1152392303, 'oneway': True, 'lanes': '3', 'name': 'Rue Réaumur', 'highway': 'secondary', 'maxspeed': '30', 'reversed': False, 'length': 69.055, 'geometry

In [16]:
pos = api.get(f'node(id:{268675130});', responseformat="csv(::lon,::lat)")
print(pos)
print(graph.nodes[268675130])

[['@lon', '@lat'], ['2.4108343', '48.8634834']]
{'y': 48.8634834, 'x': 2.4108343, 'street_count': 5}


In [10]:
print(G)

Graph with 13562 nodes and 21633 edges


In [1]:
import pickle

# save graph object t  o file
G = pickle.load(open('paris_graph_with_weights.pickle', 'rb'))

In [4]:
print(G[17348293])

{17348292: {'weight': array([19.831, 19.831, 19.831,  0.   ])}, 10068668322: {'weight': array([36.487, 36.487, 36.487,  0.   ])}, 10068668320: {'weight': array([40.277, 40.277,  0.   ,  0.   ])}, 277614019: {'weight': array([73.893, 73.893,  0.   ,  0.   ])}}
